## Aprendendo a teoria do Sha256

In [1]:
BLOCKSIZE = 64
BITSIZE = 8

In [2]:
from hash.utils_helper import RotR, ShR
from hash.attr_helper import K_256, init_state_256

In [3]:
# Lógica das etapas das 64 rodadas para gerar o hash final

rotr = lambda x, n: RotR(x, n, 32)
shr = lambda x, n: ShR(x, n, 32)
Ch = lambda x, y, z: (x & y) ^ (~x & z)
Maj = lambda x, y, z: (x & y) ^ (x & z) ^ (y & z)
sigma0 = lambda x: rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22)
sigma1 = lambda x: rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25)
gamma0 = lambda x: rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3)
gamma1 = lambda x: rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10)

In [4]:
# Mensagem que iremos usar para o hash
msg = 'abc'

In [5]:
data = [0] * BLOCKSIZE
state = list(init_state_256)
datalen = 0
bitlen = 0

In [6]:
for i in range(len(msg)):
    data[datalen] = ord(msg[i])
    datalen += 1

In [7]:
data[datalen] = 0x80
datalen += 1

In [8]:
# Preencher com zeros
data = data[:datalen] + [0] * (BLOCKSIZE - datalen)

In [9]:
bitlen = (datalen-1) * BITSIZE
n = 56
for i in range(0, 8):
    data[n + i] = bitlen >> (n - (i * 8))

In [10]:
print(data)

[97, 98, 99, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24]


In [11]:
# As 16 primeiras etapas de m é igual ao word correspondente do bloco.
m = [0] * 64
for i in range(16):
    m[i] = (data[4*i] << 24 | data[4*i+1] << 16 |
            data[4*i+2] << 8 | data[4*i+3] ) & 0xffffffff
print(m)

[1633837952, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:
for i in range(16):
    m[i] = (data[4*i] << 24 | data[4*i+1] << 16 | 
            data[4*i+2] << 8 | data[4*i+3] ) & 0xffffffff
print(m)

[1633837952, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
for i in range(16, 64):
    m[i] =( gamma1(m[i - 2]) + m[i - 7] + gamma0(m[i - 15]) + m[i - 16]) & 0xffffffff
    print(hex(m[i])[2:], end=' ')

61626380 f0000 7da86405 600003c6 3e9d7b78 183fc00 12dcbfdb e2e2c38e c8215c1a b73679a2 e5bc3909 32663c5b 9d209d67 ec8726cb 702138a4 d3b7973b 93f5997f 3b68ba73 aff4ffc1 f10a5c62 a8b3996 72af830a 9409e33e 24641522 9f47bf94 f0a64f5a 3e246a79 27333ba3 c4763f2 840abf27 7a290d5d 65c43da fb3e89cb cc7617db b9e66c34 a9993667 84badedd c21462bc 1487472c b20f7a99 ef57b9cd ebe6b238 9fe3095e 78bc8d4b a43fcf15 668b2ff8 eeaba2cc 12b1edeb 

In [14]:
a, b, c, d, e, f, g, h = state

In [15]:
for i in range(64):
    T1 = (h + sigma1(e) + Ch(e, f, g) + K_256[i] + m[i]) & 0xffffffff
    T2 = (sigma0(a) + Maj(a, b, c)) & 0xffffffff
    h = g
    g = f
    f = e
    e = (d + T1) & 0xffffffff
    d = c
    c = b
    b = a
    a = (T1 + T2) & 0xffffffff
#     print(hex(a)[2:], hex(b)[2:], hex(c)[2:], 
#           hex(d)[2:], hex(e)[2:], hex(f)[2:], 
#           hex(g)[2:], hex(h)[2:])

In [16]:
ss = (a, b, c, d, e, f, g, h)
for i in range(len(state)):
    state[i] = state[i] + ss[i]

In [17]:
''.join([hex(s)[2:] for s in state])

'ba7816bf18f01cfea414140de15dae2223b00361a3196177a9cb410ff61f20015ad'